## Old Functions and Imports

In [ ]:
%load_ext lab_black

import h5py
import os

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm

from typing import Dict, List, Optional, Tuple
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    cnn,
)

colors = pc.qualitative.D3
colors_seq = pc.sequential.dense
angstrom = "\u212B"
Theta = "\u0398"

In [ ]:
def average_timescans(files, ddir, new_filename):
    datas = []
    for i in range(0, len(files)):
        ad = ARPES_DATA[files[i]]
        datas.append(ad.data)
    data_avg = np.mean(datas, axis=0)
    print(data_avg.shape)

    new_data = data_avg

    new_fn = os.path.join(ddir, new_filename)

    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data.T
        axes_names = [
            "angles",
            "energies",
        ]  # Change these to match your axes labels
        axes = [ad.theta, ad.energy]
        for axis, name in zip(axes, axes_names):
            f[name] = np.atleast_2d(axis).T
        entry_group = f.require_group("entry1")
        entry_group["ScanValues"] = np.atleast_2d(ad.phi_or_time).T
    return new_fn

# Averaging Datasets

In [ ]:
ddir = r"E:\atully\arpes_data\2023_May\May13\C60\TR"
files = []
## TR3
# files = [f"TR3\TR_003_{i}.h5" for i in range(1, 6)]

## TR4
files = [f"TR4\TR_004_{i}.h5" for i in range(1, 4)]

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
# average_timescans(files, ddir, "TR3_Ali_avg.h5")
average_timescans(files, ddir, "TR4_Ali_avg.h5")

# Load Data

In [ ]:
ddir = r"E:\atully\arpes_data\2023_May\May13\C60\TR"
files = []

# files = [f"TR3\TR3_Ali_avg.h5"]
files = [f"TR4\TR4_Ali_avg.h5"]

# ### February Comparison ###
# ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
# files = []
# files = ["TR3_Ali_avg.h5"]

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
ad = ARPES_DATA[files[0]]
for k in ["energy", "theta", "phi_or_time"]:
    print(f"{k}.shape = {getattr(ad, k).shape}")
print(f"Data.shape = {ad.data.shape}")

In [ ]:
time_zero = 37.79  # May13
time_zero = 37.8  # May14

# time_zero = 37.96  # February

In [ ]:
## Integrate over desired angular range ##

slice_dim = "x"
slice_val = 0
int_range = 50  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

xlim = None
ylim = None

x_bin = 1
y_bin = 1

In [ ]:
title = f"C<sub>60</sub> Pump Effect: {files[0]}"
# xaxis_title=f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]",
xaxis_title = f"Delay"
yaxis_title = f"E<sub>K</sub> (eV)"

In [ ]:
## Load Data ##

xaxis_ps = False
xaxis_ps = True

x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_dataclass=ad,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    # ylim=ylim,
    ylim=(
        ad.energy[57],
        ad.energy[1007],
    ),  # get rid of zero padding on datasets
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

# Convert mm to ps
if xaxis_ps and np.max(x_2d) > 37:
    x_2d = tr_functions.mm_to_ps(x_2d, time_zero)

## Plot Data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=600,
    width=800,
    dtick_y=0.1,
)

fig.add_trace(
    go.Heatmap(
        x=x_2d,
        y=y_2d,
        z=d_2d,
        coloraxis="coloraxis",
    )
)

# fig.update_coloraxes(colorscale="greys", showscale=False)

fig.show()

In [ ]:
## Integrated over energy ##

xlim = None
ylim = None
# ylim = (3.3, 3.545)
# ylim = (1.95, 2.41)  # looking for pump effect multiple of background
# ylim = (2, 2.08)  # looking for peak of CT1

## Get slice
x_1d, row = tr_functions.get_1d_y_slice(
    x=x_2d,
    y=y_2d,
    data=d_2d,
    xlims=xlim,
    y_range=ylim,
)

## Plot data
fig = tr_functions.thesis_fig(
    # title=f"Average over {ylim} eV",
    title=f"Average over All Energy",
    xaxis_title="Delay (mm)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

fig.add_trace(go.Scatter(x=x_1d, y=row, name="data"))

fig.show()

In [ ]:
## EDCs with different time integration ##

title = f"{files[0]}"
spacing = 0

x_plot, y_plot, z_plot = x_2d, y_2d, analysis_functions.norm_data(d_2d)  # regular

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)
else:
    x_edc = x_plot


## Normalize?
normalize = False
# normalize = True

## Logplot?
logplot = False
# logplot = True

if logplot:
    title = f"{title}<br>(logplot)"
else:
    title = f"{title}"


## Set up integration limits

xlim1 = (-2, -1)
xlim2 = (-0.5, 0.5)
xlim3 = (0.5, 2)

# ## check for pre-timezero intensity
# xlim1 = (-2, -1.5)
# xlim2 = (-1.5, -1)
# xlim3 = (-1, -0.5)
# ylim = (1.95, 2.45)

xlims = [xlim1, xlim2, xlim3]

# xlim = None
ylim = None


# Get and plot data

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
)

for i, xlim in enumerate(xlims):
    color = colors[i % len(colors)]

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=xlim,
    )

    if normalize:
        col = analysis_functions.norm_data(col)

    if logplot:
        col = np.log(col)

    # Plot Data
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=i * spacing + col,
            name=f"{np.round(xlim[0], 1), np.round(xlim[1], 2)} ps",
            line=dict(color=color),
        )
    )

fig.show()

# Difference Map

In [ ]:
## Difference Map ##
title = f"Difference Map of {files[0]}"
x, y, d = x_2d, y_2d, d_2d

d_diff = d - np.mean(d[:, 0:4], axis=1)[:, None]

## Plot Data
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
    height=400,
)

# x_plot = tr_functions.mm_to_ps(x, time_zero)
x_plot = x
y_plot = y

fig.add_trace(go.Heatmap(x=x_plot, y=y_plot, z=d_diff, coloraxis="coloraxis"))
# for h in [1.63, 1.8, 1.98]:
#     fig.add_hline(y=h, line=dict(color="black", width=1, dash="dash"))
fig.update_coloraxes(colorscale="RdBu", cmid=0, showscale=True)

# Angular Dispersion

In [ ]:
## Set up general parameters ##

## This integrates from zero delay to 1 ps
slice_center = 0.5
integration = 1

slice_val = time_zero + tr_functions.ps_to_mm(slice_center, time_zero)
int_range = tr_functions.ps_to_mm(integration)

## Slicing in time to look for angular dispersion
slice_dim = "z"

xlim = (-12, 12)  # theta
# xlim = (-0.19, 0.19)  # kx
# xlim = None
ylim = None
x_bin = 2
y_bin = 2

In [ ]:
xaxis_ps = False
xaxis_ps = True

x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_dataclass=ad,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    # ylim=ylim,
    ylim=(
        ad.energy[57],
        ad.energy[1007],
    ),  # get rid of zero padding on datasets
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

# Convert mm to ps
if xaxis_ps and np.max(x_2d) > 37:
    x_2d = tr_functions.mm_to_ps(x_2d, time_zero)

## Plot data
fig = tr_functions.thesis_fig(
    title=f"Integration range: {slice_center - integration/2} to {slice_center + integration/2} ps",
    xaxis_title=f"{Theta}",
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=600,
    width=600,
)

fig.add_trace(
    go.Heatmap(
        x=x_2d,
        y=y_2d,
        z=d_2d,
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0, cmax=0.1)

fig.show()

# Fitting EDCs

In [ ]:
## Integrate over desired angular range ##

slice_dim = "x"
slice_val = 0
int_range = 50  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

# xlim = (
#     time_zero + tr_functions.ps_to_mm(0, time_zero),
#     time_zero + tr_functions.ps_to_mm(1, time_zero),
# )
xlim = None
ylim = None

x_bin = 1
y_bin = 1

In [ ]:
## Load Data ##

xaxis_ps = False
xaxis_ps = True

x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_dataclass=ad,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    # ylim=ylim,
    ylim=(
        ad.energy[57],
        ad.energy[1007],
    ),  # get rid of zero padding on datasets
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

# Convert mm to ps
if xaxis_ps and np.max(x_2d) > 37:
    x_2d = tr_functions.mm_to_ps(x_2d, time_zero)

In [ ]:
## Plot EDC for Fit ##

xlim = (-2, -0.5)  # for EF fit
xlim = (-0.5, 1)


fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_2d,
    y=y_2d,
    data=d_2d,
    ylims=ylim,
    x_range=xlim,
)

if normalize:
    col = analysis_functions.norm_data(col)

if logplot:
    col = np.log(col)

# Plot Data
fig.add_trace(
    go.Scatter(
        x=y_1d,
        y=col,
        name=f"data",
        line=dict(color=colors[0]),
    )
)

fig.show()

In [ ]:
x, data = y_1d, col
offset_type = "constant"
T = 10.6  # measurement temp
k_B = 8.617333e-5  # eV/K


## FD
def fermi_dirac(x, center, theta, amp):
    arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
    return -amp / 2 * np.tanh(arg)


## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

# Fit for 2 peaks
gauss1 = fitting_functions.make_gaussian(
    num="A_", amplitude=1, center=2, sigma=0.1
)  # 1.86
gauss2 = fitting_functions.make_gaussian(
    num="B_", amplitude=1, center=2.4, sigma=0.1
)  # 2.1
gauss3 = fitting_functions.make_gaussian(
    num="C_", amplitude=1, center=1.8, sigma=0.1
)  # 2.1

## Full model
full_model = lm.models.Model(fermi_dirac) + offset
full_model = lm.models.Model(fermi_dirac) + gauss1 + gauss2 + gauss3 + offset

params = full_model.make_params()

# if offset_type == "constant":
#     # params = linear_params.copy()
#     params = params
#     params["b"].value = 0
#     params["b"].vary = False

params[
    "center"
].value = (
    1.876  # fix once determined by fitting from (0, 100) ps with a constant offset
)
#     params["center"].vary = False
#     # # params["c"].value = 0  # determined by fitting from (0, 100) ps with a constant offset
#     # # params["c"].vary = False
params["theta"].value = k_B * (10.6)
params["theta"].min = 0
params["amp"].value = 1
params["amp"].min = 0

params["iC__height"].max = 0.25

#     params["iA__sigma"].value = tr_functions.fwhm_to_sig(0.2)
#     # params["iA__sigma"].vary = False
#     params["iB__sigma"].value = tr_functions.fwhm_to_sig(0.1)

fit = full_model.fit(data, x=x, params=params)
fit.plot()

# fig.add_trace(
#     go.Scatter(
#         x=i * spacing + fit.eval(x=y_1d),
#         y=y_1d,
#         name="fit",
#         line=dict(color="red"),
#         opacity=0.5,
#     )
# )

components = fit.eval_components(x=y_1d)

# fig.show()

In [ ]:
# fit.params

In [ ]:
## Plot Data and Fit
fig = tr_functions.thesis_fig(
    title=f"Fit of EDC: 3 Peaks + FD",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_edc,
    y=y_plot,
    data=z_plot,
    ylims=ylim,
    x_range=xlim,
)

if logplot:
    col = np.log(col)

fig.add_trace(
    go.Scatter(
        x=y_1d,
        y=col,
        name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
        line=dict(color=colors[0]),
    )
)

fig.add_trace(go.Scatter(x=y_1d, y=fit.eval(x=y_1d), name="fit"))

for model_name, model_value in list(components.items())[1:4]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        y=fit.eval(x=fit.params[f"{model_name}center"].value),
        text=f'{fit.params[f"{model_name}center"].value:.2f} eV',
        font=dict(size=18, color=colors[0]),
        # ax=-300,
        # ay=0,
        bgcolor="white",
        # opacity=1,
        # bordercolor=colors[0],
        # borderwidth=2,
        # borderpad=4,
    )

fig.update_layout(showlegend=True)

fig.show()

print(f"Center FD = {fit.params['center'].value:.2f} eV")
print(f"T = {fit.params['theta'].value / k_B:.2f} K")
print(f"Center A = {fit.params['iA__center'].value:.2f} eV")
print(f"FWHM A = {fit.params['iA__fwhm'].value:.3f} ps")
print(f"Center B = {fit.params['iB__center'].value:.2f} eV")
print(f"FWHM B = {fit.params['iB__fwhm'].value:.3f} ps")

In [ ]:
## Plot Residuals ##
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=yaxis_title,
    yaxis_title="Residuals",
    equiv_axes=False,
    gridlines=False,
    height=400,
    width=900,
    dtick_x=0.2,
)

fig.add_trace(go.Scatter(x=y_1d, y=col - fit.eval(x=y_1d), name="fit"))
fig.add_hline(y=0, line=dict(dash="dash"))

In [ ]:
fig = tr_functions.thesis_fig(
    title=f"{title}<br> Fit Components",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

for i, xlim in enumerate(xlims):
    color = colors[i % len(colors)]


for model_name, model_value in list(components.items())[0:5]:
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=model_value,
            name=model_name,
            line=dict(color=color),
        )
    )

fig.data[4].update(name="offset")

for model_name, model_value in list(components.items())[1:4]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        yref="y domain",
        y=0.3,
        showarrow=False,
        text=f'Center: {fit.params[f"{model_name}center"].value:.2f} eV<br>FWHM: {fit.params[f"{model_name}fwhm"].value:.2f} eV',
        font=dict(size=18, color=colors[0]),
        bgcolor="white",
    )


fig.show()